# ADIOS2-Python-Full-API-NoMPI: Write Variable Shapes

## Writer: Generate Variables with different shapes

In [ ]:
import numpy as np
import adios2


# Initialize dimensions
nx = 10
nsteps = 3

# Global array dimensions 
shape = [nx]
start = [0]
count = [nx]


# initialize ADIOS2
adios = adios2.ADIOS()
io = adios.DeclareIO("VariableShapes")

# dummy np arrays to inform adios2 of the type
# Global values
var_nsteps = io.DefineVariable("nsteps", np.array([0], dtype=np.uint32))
var_size = io.DefineVariable("size", np.array([0], dtype=np.uint64))
var_step = io.DefineVariable("step") # string type
# Local values
var_rank = io.DefineVariable("rank", np.array([0], dtype=np.uint32), [adios2.LocalValueDim])
# Global array
# global array, there is a total shape, 
# and a local "window" for each rank (start and count)
var_global_array = io.DefineVariable("GlobalArray", np.array([0], dtype=np.float64), shape, start, count)
# Local array
# local array, no shape, no start offset
# only a local "dimension" for each rank (count)
var_local_array = io.DefineVariable("LocalArray", np.array([0], dtype=np.float32), [], [], count)

io.DefineAttribute("Greeting", "Hello ADIOS Variables")

fw = io.Open("variable_shapes.bp", adios2.Mode.Write)

for step in range(0,nsteps):
    
    fw.BeginStep()
    
    #global variable that doesn't change
    #save nsteps as a numpy array
    if(step == 0):
        fw.Put(var_nsteps, np.array(nsteps, dtype=np.uint32) )
        fw.Put(var_size, np.array(1, dtype=np.uint64))
    
    #global variable that changes over time
    #save step as a string
    fw.Put(var_step, str(step))
    
    # local values are independent quantities must be explicit
    # we can search for a variable name inside IO
    fw.Put(var_rank, np.array(0, dtype=np.uint32) )
    fw.Put(var_global_array, np.arange(step, nx+step, dtype=np.float64))
    fw.Put(var_local_array, np.arange(step, nx+step, dtype=np.float32))
    
    fw.EndStep()
    

# Close is always a good practice
fw.Close()

## Understand the output with bpls 

In [ ]:
!bpls variable_shapes.bp -lavd -n 10